In [1]:
import signal
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D

c:\Users\tanpe\anaconda3\envs\personal\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\tanpe\anaconda3\envs\personal\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\tanpe\anaconda3\envs\personal\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
pretrained_model = ResNet50V2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(112,112,3)
)

unfreezed = [
    'conv4_block5_1_conv',
    'conv4_block5_2_conv',
    'conv4_block5_3_conv',
    'conv4_block6_1_conv',
    'conv4_block6_2_conv',
    'conv4_block6_3_conv',
    'conv5_block1_1_conv',
    'conv5_block1_2_conv',
    'conv5_block1_3_conv',
    'conv5_block2_1_conv',
    'conv5_block2_2_conv',
    'conv5_block2_3_conv',
    'conv5_block3_1_conv',
    'conv5_block3_2_conv',
    'conv5_block3_3_conv',
    ]


for layer in pretrained_model.layers:
    
    if layer.name in unfreezed:
        layer.trainable = True
    else:
        layer.trainable = False

In [4]:
no_glasses_array = np.load('./data/T81-855_glasses_dataset/no_glasses.npy')
glasses_array = np.load('./data/T81-855_glasses_dataset/glasses.npy')
generated_glasses_1 = np.load('./data/T81-855_glasses_dataset/generated_glasses_1.npy')
generated_glasses_2 = np.load('./data/T81-855_glasses_dataset/generated_glasses_2.npy')

random_arrays_indices = np.random.choice(len(generated_glasses_1),int(len(generated_glasses_1)/1),replace=False)
generated_glasses_1 = generated_glasses_1[random_arrays_indices]

random_arrays_indices = np.random.choice(len(generated_glasses_2),int(len(generated_glasses_2)/2),replace=False)
generated_glasses_2 = generated_glasses_2[random_arrays_indices]

In [ ]:
feature_array = np.concatenate(
    [
        no_glasses_array,
        glasses_array,
        generated_glasses_1,
        generated_glasses_2
    ],
    axis=0
) 

feature_array = preprocess_input(feature_array)

target_array = np.concatenate([
    np.zeros(2200),
    np.ones(2200),
    np.ones(len(generated_glasses_1)+len(generated_glasses_2))*2
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    feature_array,
    target_array,
    test_size=0.05,
    stratify=target_array,
    random_state=42
)

In [ ]:
model = tf.keras.models.Sequential([
    pretrained_model,
    GlobalAveragePooling2D(),    
    Dense(512,activation="relu"),
    Dropout(0.7),
    Dense(128,activation="relu"),
    Dropout(0.3), 
    Dense(32,activation="tanh"),
    Dropout(0.2), 
    Dense(3, activation='softmax')
])

model.compile(optimizer=RMSprop(learning_rate=3e-5),
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
class KeyboardInterruptCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.original_sigint_handler = None
        self._training_pid = None
        self.best_epoch = None
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.original_sigint_handler = signal.signal(signal.SIGINT, self.interrupt_training)
        self._training_pid = os.getpid()
        self.best_val_loss = float('inf')

    def on_train_end(self, logs=None):
        signal.signal(signal.SIGINT, self.original_sigint_handler)
        self.original_sigint_handler = None
        self._training_pid = None

    def on_epoch_end(self, epoch, logs=None):
        if logs['val_loss'] < self.best_val_loss:
            self.best_epoch = epoch
            self.best_val_loss = logs['val_loss']
            self.best_weights = self.model.get_weights()
        
        logs['best_val_loss'] = self.best_val_loss

    def interrupt_training(self, signum, frame):
        if self._training_pid == os.getpid():
            print(f"Keyboard interrupt detected. Restoring weights from Epoch {self.best_epoch+1}")
            self.model.set_weights(self.best_weights)
            self.model.stop_training = True            

keyboard_interrupt_callback = KeyboardInterruptCallback()

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=10,
    verbose=1,
    restore_best_weights=True,
    monitor='val_loss'
)

callbacks=[keyboard_interrupt_callback, early_stopping_callback]

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=200, 
    batch_size=32,
    validation_data=(X_test,y_test),
    callbacks=callbacks
)

In [ ]:
model.save('models/discriminator')
model.layers[0].save('models/discriminator_resnet')